In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
from IPython.display import display, HTML, clear_output
import seaborn as sns
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from math import sqrt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, Concatenate, Flatten, LSTM, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import top_k_categorical_accuracy, categorical_accuracy
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report, log_loss

# Con multiclasificador

In [110]:
path = r'C:\Users\ing_l\Tesis grado\Data\Predictors data\Dataset2019_multiclassifier.csv'

In [111]:
data = pd.read_csv(path)
tiempo = 'media hora'
#data['media hora'] = data['media hora'] / (1800*2)
#data = data.loc[(data['media hora'] >= 9) & (data['media hora'] <= 20)]
data[0:50]

,dia de la semana,fecha,id_cuadra,media hora,mes,ocupacion,porcentaje ocupacion,target
0,2,2019-01-02,0,10.0,1,2,0,0
1,2,2019-01-02,0,10.5,1,1,0,0
2,2,2019-01-02,0,11.0,1,5,0,0
3,2,2019-01-02,0,11.5,1,4,0,0
4,2,2019-01-02,0,12.0,1,2,0,0
5,2,2019-01-02,0,12.5,1,3,0,0
6,2,2019-01-02,0,13.0,1,4,0,0
7,2,2019-01-02,0,13.5,1,3,0,1
8,2,2019-01-02,0,14.0,1,6,1,1
9,2,2019-01-02,0,14.5,1,6,1,0


### Acomodo los targets

In [112]:
'''out = pd.DataFrame(columns=data.columns.values)
out['target'] = 0

for s in data['id_cuadra'].drop_duplicates().sort_values():
    clear_output()
    print('calle:', s)
    
    data_s = data.loc[data['id_cuadra'] == s].copy()
    
    for f in data_s['fecha'].drop_duplicates():
        
        data_f = data_s.loc[data_s['fecha'] == f].sort_values(by=tiempo).reset_index(drop=True)
        
        to_append = pd.DataFrame(columns=data.columns.values)
        to_append['target'] = 0
        
        to_append = to_append.append(data_f[:-1])
        #display(to_append)
        to_append['target'] = data_f['porcentaje ocupacion'][1:].reset_index(drop=True)
        #display(data_f['porcentaje ocupacion'][1:].reset_index(drop=True))
        out = out.append(to_append)
        '''

"out = pd.DataFrame(columns=data.columns.values)\nout['target'] = 0\n\nfor s in data['id_cuadra'].drop_duplicates().sort_values():\n    clear_output()\n    print('calle:', s)\n    \n    data_s = data.loc[data['id_cuadra'] == s].copy()\n    \n    for f in data_s['fecha'].drop_duplicates():\n        \n        data_f = data_s.loc[data_s['fecha'] == f].sort_values(by=tiempo).reset_index(drop=True)\n        \n        to_append = pd.DataFrame(columns=data.columns.values)\n        to_append['target'] = 0\n        \n        to_append = to_append.append(data_f[:-1])\n        #display(to_append)\n        to_append['target'] = data_f['porcentaje ocupacion'][1:].reset_index(drop=True)\n        #display(data_f['porcentaje ocupacion'][1:].reset_index(drop=True))\n        out = out.append(to_append)\n        "

In [113]:
#out

### veo si las clases estan balanceadas

In [114]:
balanced_classes = data[['porcentaje ocupacion','ocupacion']].groupby('porcentaje ocupacion').count()
balanced_classes.rename(columns={'ocupacion': 'count'}, inplace=True)
total_data = balanced_classes['count'].sum()
balanced_classes

,count
porcentaje ocupacion,
0,151448
1,171976
2,90890
3,28247
4,5905


### Separo en train y test

In [115]:
input_features = ['porcentaje ocupacion', 'ocupacion', tiempo]

In [116]:
x_train, x_test, y_train, y_test = train_test_split(data[['ocupacion',tiempo,'id_cuadra', 'dia de la semana', 'mes', 'porcentaje ocupacion']], data['target'], test_size=0.1, shuffle=True, stratify=data['target'])   

In [117]:
y_train_no_cat = y_train.copy()
y_test_no_cat = y_test.copy()

In [118]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [119]:
class_weight = compute_class_weight('balanced', np.unique(y_train_no_cat), y_train_no_cat)
class_weight

array([ 0.60008326,  0.52031867,  0.97521957,  3.1306496 , 14.96270621])

# Modelo baseline

In [120]:
balanced_accuracy_score(y_test_no_cat, x_test['porcentaje ocupacion'])

0.5758612885412144

In [121]:
y_test_no_cat.unique()

array([2, 1, 0, 4, 3], dtype=int64)

In [122]:
cnf_matrix = confusion_matrix(y_test_no_cat, x_test['porcentaje ocupacion'],labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[10952,  3707,   258,    27,     3],
       [ 3796, 10503,  2715,   207,    17],
       [  326,  2779,  5023,  1016,    53],
       [   20,   249,  1017,  1388,   191],
       [    9,    24,    64,   199,   304]], dtype=int64)

In [123]:
print(classification_report(y_test_no_cat, x_test['porcentaje ocupacion']))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73     14947
           1       0.61      0.61      0.61     17238
           2       0.55      0.55      0.55      9197
           3       0.49      0.48      0.49      2865
           4       0.54      0.51      0.52       600

    accuracy                           0.63     44847
   macro avg       0.58      0.58      0.58     44847
weighted avg       0.63      0.63      0.63     44847



# Naive bayes classifier

### Multinomial

In [38]:
model = MultinomialNB()

model.fit(x_train[input_features], y_train_no_cat)

model.score(x_test[input_features], y_test_no_cat)

0.583004437309073

In [39]:
cnf_matrix = confusion_matrix(y_test_no_cat, model.predict(x_test[input_features]),labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11423,  3402,   122,     0,     0],
       [ 4561,  9881,  2793,     3,     0],
       [  651,  3808,  4657,    81,     0],
       [   98,   759,  1823,   185,     0],
       [   15,   137,   347,   101,     0]], dtype=int64)

### Gaussian

In [19]:
model = GaussianNB()

model.fit(x_train[input_features], y_train_no_cat)

model.score(x_test[input_features], y_test_no_cat)

0.6173211140098558

In [20]:
cnf_matrix = confusion_matrix(y_test_no_cat, model.predict(x_test[input_features]),labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11617,  3192,   131,     7,     0],
       [ 4715,  9991,  2457,    73,     2],
       [  625,  2960,  5005,   600,     7],
       [   89,   483,  1231,   991,    71],
       [   14,    42,   140,   323,    81]], dtype=int64)

# Arbol de decision

In [40]:
max_depth = 20

best_score = np.zeros((max_depth+1, 2))
best_depth = 0
best_model = 0


for i in range(1, max_depth+1):
    clear_output()
    print("Profundidad:", i)
    dt = DecisionTreeClassifier(max_depth=i, class_weight='balanced')
    dt = dt.fit(x_train[input_features], y_train_no_cat)
    
    if dt.score(x_test[input_features], y_test_no_cat) > max([x[1] for x in best_score]):
        best_depth = i
        best_model = dt
        
    best_score[i][0] = dt.score(x_train[input_features], y_train_no_cat)
    best_score[i][1] = dt.score(x_test[input_features], y_test_no_cat)

    
print('Mejor profundidad:', best_depth)
print('Mejor accuracy:', max([x[1] for x in best_score]))

Profundidad: 20
Mejor profundidad: 14
Mejor accuracy: 0.6312796842598167


In [41]:
cnf_matrix = confusion_matrix(y_test_no_cat, best_model.predict(x_test[input_features]),labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11815,  2699,   268,   125,    40],
       [ 3980,  9556,  2921,   642,   139],
       [  314,  2114,  5010,  1491,   268],
       [   33,   120,   727,  1527,   458],
       [    5,    10,    23,   159,   403]], dtype=int64)

In [42]:
print(classification_report(y_test_no_cat, best_model.predict(x_test[input_features])))

              precision    recall  f1-score   support

           0       0.73      0.79      0.76     14947
           1       0.66      0.55      0.60     17238
           2       0.56      0.54      0.55      9197
           3       0.39      0.53      0.45      2865
           4       0.31      0.67      0.42       600

    accuracy                           0.63     44847
   macro avg       0.53      0.62      0.56     44847
weighted avg       0.64      0.63      0.63     44847



# Random forest

In [22]:
max_depth = 20

best_score = np.zeros((max_depth+1, 2))
best_depth = 0
best_model = 0


for i in range(1, max_depth+1):
    clear_output()
    print("Profundidad:", i)
    dt = RandomForestClassifier(max_depth=i, class_weight='balanced')
    dt = dt.fit(x_train[input_features], y_train_no_cat)
    
    if dt.score(x_test[input_features], y_test_no_cat) > max([x[1] for x in best_score]):
        best_depth = i
        best_model = dt
        
    best_score[i][0] = dt.score(x_train[input_features], y_train_no_cat)
    best_score[i][1] = dt.score(x_test[input_features], y_test_no_cat)

    
print('Mejor profundidad:', best_depth)
print('Mejor accuracy:', max([x[1] for x in best_score]))

Profundidad: 20
Mejor profundidad: 6
Mejor accuracy: 0.6337101701340112


In [23]:
cnf_matrix = confusion_matrix(y_test_no_cat, best_model.predict(x_test[input_features]),labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11761,  2914,   172,    70,    30],
       [ 4027,  9725,  2901,   460,   125],
       [  432,  2270,  5094,  1169,   232],
       [   50,   161,   814,  1434,   406],
       [    9,    12,    29,   144,   406]], dtype=int64)

In [24]:
print(classification_report(y_test_no_cat, best_model.predict(x_test[input_features])))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75     14947
           1       0.64      0.56      0.60     17238
           2       0.57      0.55      0.56      9197
           3       0.44      0.50      0.47      2865
           4       0.34      0.68      0.45       600

    accuracy                           0.63     44847
   macro avg       0.54      0.62      0.57     44847
weighted avg       0.64      0.63      0.63     44847



# Redes neuronales

### Una especie de regresion logistica con 5 salidas solo num

In [15]:
i_features = Input(shape=(len(input_features)))

o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(i_features)

model = Model(inputs=i_features, outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
Output (Dense)               (None, 5)                 20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


In [16]:
h = model.fit(x_train[input_features], y_train, epochs=10, batch_size=64, class_weight=class_weight, validation_data=[x_test[input_features],y_test])

Train on 403619 samples, validate on 44847 samples
Epoch 1/10
403619/403619 [==============================] - 6s 15us/sample - loss: 1.1054 - categorical_accuracy: 0.5716 - val_loss: 0.9110 - val_categorical_accuracy: 0.6103
Epoch 2/10
403619/403619 [==============================] - 6s 15us/sample - loss: 0.9018 - categorical_accuracy: 0.6162 - val_loss: 0.8961 - val_categorical_accuracy: 0.6169
Epoch 3/10
403619/403619 [==============================] - 6s 15us/sample - loss: 0.8929 - categorical_accuracy: 0.6192 - val_loss: 0.8890 - val_categorical_accuracy: 0.6193
Epoch 4/10
403619/403619 [==============================] - 6s 15us/sample - loss: 0.8889 - categorical_accuracy: 0.6203 - val_loss: 0.8873 - val_categorical_accuracy: 0.6186
Epoch 5/10
403619/403619 [==============================] - 6s 16us/sample - loss: 0.8864 - categorical_accuracy: 0.6209 - val_loss: 0.8852 - val_categorical_accuracy: 0.6202s - loss: 0.8868 - 
Epoch 6/10
403619/403619 [=============================

In [17]:
pred = [x.argmax() for x in model.predict(x_test[input_features])]

cnf_matrix = confusion_matrix(y_test_no_cat, pred,labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11142,  3597,   176,    31,     1],
       [ 4097, 10867,  2123,   151,     0],
       [  428,  3220,  4910,   636,     3],
       [   52,   367,  1460,   979,     7],
       [    9,    27,   127,   426,    11]], dtype=int64)

In [18]:
print(classification_report(y_test_no_cat, pred))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73     14947
           1       0.60      0.63      0.62     17238
           2       0.56      0.53      0.55      9197
           3       0.44      0.34      0.38      2865
           4       0.50      0.02      0.04       600

    accuracy                           0.62     44847
   macro avg       0.56      0.45      0.46     44847
weighted avg       0.62      0.62      0.62     44847



### La misma especie de regresio logistica con 5 salidas pero con valores categoricos tambien

In [29]:
i_features = Input(shape=(len(input_features)))

max_street=max((data['id_cuadra'].drop_duplicates()))

i_street = Input(shape=(1))
emb_street = Embedding(input_dim=max_street+1, output_dim=10)(i_street)
flat_street = Flatten()(emb_street)

i_weekday = Input(shape=(1))
emb_weekday = Embedding(input_dim=7, output_dim=5)(i_weekday)
flat_weekday = Flatten()(emb_weekday)

i_month = Input(shape=(1))
emb_month = Embedding(input_dim=12+1, output_dim=5)(i_month)
flat_month = Flatten()(emb_month)

concat = Concatenate(axis=-1)
d = concat([i_features, flat_street])
d = concat([d, flat_weekday])
d = concat([d, flat_month])

o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(d)

model = Model(inputs=[i_features,i_street,i_weekday, i_month], outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 10)        1910        input_3[0][0]                    
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
____________________________________________________________________________________________

In [30]:
h = model.fit([x_train[input_features],x_train['id_cuadra'], x_train['dia de la semana'], x_train['mes']], y_train, epochs=10, batch_size=64, class_weight=class_weight, validation_data=[[x_test[input_features],x_test['id_cuadra'], x_test['dia de la semana'], x_test['mes']],y_test])

Train on 403619 samples, validate on 44847 samples
Epoch 1/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.9276 - categorical_accuracy: 0.6120 - val_loss: 0.8292 - val_categorical_accuracy: 0.6341
Epoch 2/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8181 - categorical_accuracy: 0.6398 - val_loss: 0.8179 - val_categorical_accuracy: 0.6387
Epoch 3/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8128 - categorical_accuracy: 0.6418 - val_loss: 0.8166 - val_categorical_accuracy: 0.6366
Epoch 4/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8114 - categorical_accuracy: 0.6424 - val_loss: 0.8152 - val_categorical_accuracy: 0.6402
Epoch 5/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8107 - categorical_accuracy: 0.6428 - val_loss: 0.8142 - val_categorical_accuracy: 0.6401
Epoch 6/10
403619/403619 [==============================] - 8s 21us/sample

In [31]:
pred = [x.argmax() for x in model.predict([x_test[input_features],x_test['id_cuadra'], x_test['dia de la semana'], x_test['mes']])]

cnf_matrix = confusion_matrix(y_test_no_cat, pred,labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11106,  3711,   105,    21,     4],
       [ 3699, 11699,  1702,   120,    18],
       [  291,  3823,  4571,   489,    23],
       [   24,   470,  1161,  1166,    44],
       [    4,    38,    70,   319,   169]], dtype=int64)

In [32]:
print(classification_report(y_test_no_cat, pred))

              precision    recall  f1-score   support

           0       0.73      0.74      0.74     14947
           1       0.59      0.68      0.63     17238
           2       0.60      0.50      0.54      9197
           3       0.55      0.41      0.47      2865
           4       0.66      0.28      0.39       600

    accuracy                           0.64     44847
   macro avg       0.63      0.52      0.56     44847
weighted avg       0.64      0.64      0.64     44847



In [33]:
'''comp = pd.DataFrame([y_t, pred]).T
comp.rename({0: 'y',
             1: 'yhat'}, axis=1, inplace=True)
comp[0:50]'''

"comp = pd.DataFrame([y_t, pred]).T\ncomp.rename({0: 'y',\n             1: 'yhat'}, axis=1, inplace=True)\ncomp[0:50]"

### red neuronal Solo var num 

In [34]:
i_features = Input(shape=(len(input_features)))
d = Dense(64, activation='relu')(i_features)
d = Dropout(0.3)(d)
d = Dense(64, activation='relu')(d)
o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(d)

model = Model(inputs=i_features, outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                256       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
Output (Dense)               (None, 5)                 325       
Total params: 4,741
Trainable params: 4,741
Non-trainable params: 0
_________________________________________________________________


In [35]:
h = model.fit(x_train[input_features], y_train, epochs=10, batch_size=64, class_weight=class_weight, validation_data=[x_test[input_features],y_test])

Train on 403619 samples, validate on 44847 samples
Epoch 1/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.9085 - categorical_accuracy: 0.6132 - val_loss: 0.8669 - val_categorical_accuracy: 0.6271
Epoch 2/10
403619/403619 [==============================] - 8s 20us/sample - loss: 0.8656 - categorical_accuracy: 0.6302 - val_loss: 0.8587 - val_categorical_accuracy: 0.6332
Epoch 3/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8545 - categorical_accuracy: 0.6341 - val_loss: 0.8522 - val_categorical_accuracy: 0.6352
Epoch 4/10
403619/403619 [==============================] - 8s 19us/sample - loss: 0.8485 - categorical_accuracy: 0.6365 - val_loss: 0.8422 - val_categorical_accuracy: 0.6378
Epoch 5/10
403619/403619 [==============================] - 8s 21us/sample - loss: 0.8423 - categorical_accuracy: 0.6396 - val_loss: 0.8410 - val_categorical_accuracy: 0.6392
Epoch 6/10
403619/403619 [==============================] - 8s 20us/sample

In [36]:
pred = [x.argmax() for x in model.predict(x_test[input_features])]

cnf_matrix = confusion_matrix(y_test_no_cat, pred,labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11201,  3551,   186,     9,     0],
       [ 3445, 11227,  2502,    64,     0],
       [  313,  3073,  5454,   357,     0],
       [   36,   273,  1657,   897,     2],
       [    5,    21,   183,   382,     9]], dtype=int64)

In [37]:
print(classification_report(y_test_no_cat, best_model.predict(x_test[input_features])))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75     14947
           1       0.64      0.56      0.60     17238
           2       0.57      0.55      0.56      9197
           3       0.44      0.50      0.47      2865
           4       0.34      0.68      0.45       600

    accuracy                           0.63     44847
   macro avg       0.54      0.62      0.57     44847
weighted avg       0.64      0.63      0.63     44847



In [38]:
'''comp = pd.DataFrame([y_t, pred]).T
comp.rename({0: 'y',
             1: 'yhat'}, axis=1, inplace=True)
comp[0:50]'''

"comp = pd.DataFrame([y_t, pred]).T\ncomp.rename({0: 'y',\n             1: 'yhat'}, axis=1, inplace=True)\ncomp[0:50]"

### Red neuronal Var num y cat 

In [124]:
i_features = Input(shape=(len(input_features)))

max_street=max((data['id_cuadra'].drop_duplicates()))

i_street = Input(shape=(1))
emb_street = Embedding(input_dim=max_street+1, output_dim=10)(i_street)
flat_street = Flatten()(emb_street)

i_weekday = Input(shape=(1))
emb_weekday = Embedding(input_dim=7, output_dim=5)(i_weekday)
flat_weekday = Flatten()(emb_weekday)

i_month = Input(shape=(1))
emb_month = Embedding(input_dim=12+1, output_dim=5)(i_month)
flat_month = Flatten()(emb_month)

concat = Concatenate(axis=-1)
d = concat([i_features, flat_street])
d = concat([d, flat_weekday])
d = concat([d, flat_month])

d = BatchNormalization()(d)
d = Dense(256, activation='relu')(d)
d = Dropout(0.2)(d)

o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(d)

model = Model(inputs=[i_features,i_street,i_weekday, i_month], outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 1, 10)        1910        input_25[0][0]                   
__________________________________________________________________________________________________
input_26 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 3)]          0                                            
___________________________________________________________________________________________

In [125]:
h = model.fit([x_train[input_features],x_train['id_cuadra'], x_train['dia de la semana'], x_train['mes']], y_train, epochs=10, batch_size=64, class_weight=class_weight, validation_data=[[x_test[input_features],x_test['id_cuadra'], x_test['dia de la semana'], x_test['mes']],y_test])

Train on 403619 samples, validate on 44847 samples
Epoch 1/10
403619/403619 [==============================] - 17s 42us/sample - loss: 0.8364 - categorical_accuracy: 0.6309 - val_loss: 0.7811 - val_categorical_accuracy: 0.6517
Epoch 2/10
403619/403619 [==============================] - 18s 44us/sample - loss: 0.7997 - categorical_accuracy: 0.6462 - val_loss: 0.7576 - val_categorical_accuracy: 0.6636
Epoch 3/10
403619/403619 [==============================] - 14s 35us/sample - loss: 0.7933 - categorical_accuracy: 0.6479 - val_loss: 0.7528 - val_categorical_accuracy: 0.6631
Epoch 4/10
403619/403619 [==============================] - 14s 34us/sample - loss: 0.7893 - categorical_accuracy: 0.6498 - val_loss: 0.7482 - val_categorical_accuracy: 0.6663
Epoch 5/10
403619/403619 [==============================] - 14s 36us/sample - loss: 0.7861 - categorical_accuracy: 0.6523 - val_loss: 0.7427 - val_categorical_accuracy: 0.6712
Epoch 6/10
403619/403619 [==============================] - 13s 31us/

In [126]:
pred = [x.argmax() for x in model.predict([x_test[input_features],x_test['id_cuadra'], x_test['dia de la semana'], x_test['mes']])]

cnf_matrix = confusion_matrix(y_test_no_cat, pred,labels=np.sort(y_test_no_cat.unique()))
cnf_matrix

array([[11510,  3333,    89,    14,     1],
       [ 3189, 12216,  1689,   138,     6],
       [  173,  3445,  4766,   796,    17],
       [    5,   349,  1053,  1406,    52],
       [    1,    35,    81,   278,   205]], dtype=int64)

In [127]:
print(classification_report(y_test_no_cat, pred))

              precision    recall  f1-score   support

           0       0.77      0.77      0.77     14947
           1       0.63      0.71      0.67     17238
           2       0.62      0.52      0.56      9197
           3       0.53      0.49      0.51      2865
           4       0.73      0.34      0.47       600

    accuracy                           0.67     44847
   macro avg       0.66      0.57      0.60     44847
weighted avg       0.67      0.67      0.67     44847



In [128]:
'''comp = pd.DataFrame([y_t, pred]).T
comp.rename({0: 'y',
             1: 'yhat'}, axis=1, inplace=True)
comp[0:50]'''

"comp = pd.DataFrame([y_t, pred]).T\ncomp.rename({0: 'y',\n             1: 'yhat'}, axis=1, inplace=True)\ncomp[0:50]"

# LSTM

In [15]:
#['porcentaje ocupacion','ocupacion','media hora', 'id_cuadra']
train_size = 3

ocup =[]
porc_ocup =[]
media_hora =[]
mes =[]
dia_de_la_semana =[]
id_cuadra =[]
target =[]
for s in data['id_cuadra'].drop_duplicates().sort_values():
    clear_output()
    print('calle:', s)
    
    d_s = data.loc[data['id_cuadra'] == s].copy()
    
    for f in d_s['fecha'].drop_duplicates():
        d_f = d_s.loc[d_s['fecha'] == f]
        
        for i in range(len(d_f) - train_size):
            ocup.append(d_f['ocupacion'].iloc[i:i+train_size])
            
            porc_ocup.append(d_f['porcentaje ocupacion'].iloc[i+train_size])
            media_hora.append(d_f[tiempo].iloc[i+train_size])
            mes.append(d_f['mes'].iloc[i+train_size])
            dia_de_la_semana.append(d_f['dia de la semana'].iloc[i+train_size])
            id_cuadra.append(d_f['id_cuadra'].iloc[i+train_size])
            target.append(d_f['target'].iloc[i+train_size])

calle: 190


In [16]:
from random import shuffle

to_shuffle = list(zip(ocup, porc_ocup, media_hora, mes, dia_de_la_semana, id_cuadra, target))
shuffle(to_shuffle)
ocup, porc_ocup, media_hora, mes, dia_de_la_semana, id_cuadra, target = zip(*to_shuffle)

In [17]:
train_test = int(len(porc_ocup) * 0.1)

train_ocup = ocup[train_test:]
train_porc_ocup = porc_ocup[train_test:]
train_media_hora = media_hora[train_test:]
train_mes = mes[train_test:]
train_dia_de_la_semana = dia_de_la_semana[train_test:]
train_id_cuadra = id_cuadra[train_test:]
train_target = target[train_test:]

test_ocup = ocup[:train_test]
test_porc_ocup = porc_ocup[:train_test]
test_media_hora = media_hora[:train_test]
test_mes = mes[:train_test]
test_dia_de_la_semana = dia_de_la_semana[:train_test]
test_id_cuadra = id_cuadra[:train_test]
test_target = target[:train_test]

In [18]:
#porc_ocup = to_categorical(porc_ocup)
train_target = to_categorical(train_target,num_classes=max(target)+1)
test_target = to_categorical(test_target,num_classes=max(target)+1)

# LSTM num

In [97]:
#porc ocup
i_porc_ocup = Input(shape=(1), name='i_porc_ocup')

#media hora
i_time = Input(shape=(1), name='i_time')

#ocup
i_ocup = Input(shape=(train_size,1), name='i_ocup')
lstm = LSTM(10, return_sequences=False, name='lstm')(i_ocup)


concat = Concatenate(axis=-1, name='input_merge')
input_merge = concat([i_porc_ocup, i_time])
input_merge = concat([input_merge, lstm])

d = BatchNormalization()(input_merge)
d = Dense(256, activation='relu')(d)
d = Dropout(0.2)(d)
d = Dense(512, activation='relu')(d)
d = BatchNormalization()(d)
d = Dropout(0.2)(d)
d = Dense(512, activation='relu')(d)

o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(d)

model = Model(inputs=[i_porc_ocup, i_time, i_ocup], outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
i_porc_ocup (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
i_time (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
i_ocup (InputLayer)             [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
input_merge (Concatenate)       multiple             0           i_porc_ocup[0][0]                
                                                                 i_time[0][0]              

In [98]:
train_ocup_tf = tf.convert_to_tensor(np.array(train_ocup).reshape(-1,train_size,1), np.float64)
test_ocup_tf = tf.convert_to_tensor(np.array(test_ocup).reshape(-1,train_size,1), np.float64)

In [99]:
x_train = [np.array(train_porc_ocup), np.array(train_media_hora), train_ocup_tf]
x_test = [np.array(test_porc_ocup), np.array(test_media_hora), test_ocup_tf]
#xt = [x_test_porc_ocup, x_test['id_cuadra'], x_test['media hora'], x_train['ocupacion']]
h = model.fit(x_train, train_target, epochs=10, batch_size=128, class_weight=class_weight, validation_data=[x_test, test_target])

Train on 330459 samples, validate on 36717 samples
Epoch 1/10
330459/330459 [==============================] - 38s 115us/sample - loss: 0.8506 - categorical_accuracy: 0.6432 - val_loss: 0.8151 - val_categorical_accuracy: 0.6533
Epoch 2/10
330459/330459 [==============================] - 33s 100us/sample - loss: 0.8205 - categorical_accuracy: 0.6550 - val_loss: 0.8080 - val_categorical_accuracy: 0.6588
Epoch 3/10
330459/330459 [==============================] - 34s 102us/sample - loss: 0.8139 - categorical_accuracy: 0.6560 - val_loss: 0.8049 - val_categorical_accuracy: 0.6569
Epoch 4/10
330459/330459 [==============================] - 32s 96us/sample - loss: 0.8112 - categorical_accuracy: 0.6575 - val_loss: 0.8035 - val_categorical_accuracy: 0.6577
Epoch 5/10
330459/330459 [==============================] - 31s 94us/sample - loss: 0.8092 - categorical_accuracy: 0.6582 - val_loss: 0.8027 - val_categorical_accuracy: 0.6593
Epoch 6/10
330459/330459 [==============================] - 31s 94

In [100]:
x_test = [tf.convert_to_tensor(x, np.float64) for x in x_test]

In [101]:
pred = [x.argmax() for x in model.predict(x_test)]
test_t = pd.Series([x.argmax() for x in test_target])

cnf_matrix = confusion_matrix(test_t, pred,labels=np.sort(test_t.unique()))
cnf_matrix

array([[9411, 3265,  118,    4,    2],
       [2688, 9421, 1642,   66,   21],
       [ 127, 2582, 4138,  476,   37],
       [  11,  237,  867,  996,  168],
       [   2,   18,   45,  114,  261]], dtype=int64)

In [102]:
print(classification_report(test_t, pred))

              precision    recall  f1-score   support

           0       0.77      0.74      0.75     12800
           1       0.61      0.68      0.64     13838
           2       0.61      0.56      0.58      7360
           3       0.60      0.44      0.51      2279
           4       0.53      0.59      0.56       440

    accuracy                           0.66     36717
   macro avg       0.62      0.60      0.61     36717
weighted avg       0.66      0.66      0.66     36717



# LSTM num y cat

In [103]:
#porc ocup
i_porc_ocup = Input(shape=(1), name='i_porc_ocup')

#cuadra
max_street=max((data['id_cuadra'].drop_duplicates()))

i_street = Input(shape=(1), name='i_street')
emd_street = Embedding(input_dim=max_street+1, output_dim=10, name='emd_street')(i_street)
flat_street = Flatten(name='flat_street')(emd_street)

#dia de la semana
i_weekday = Input(shape=(1))
emb_weekday = Embedding(input_dim=7, output_dim=5)(i_weekday)
flat_weekday = Flatten()(emb_weekday)

#mes
i_month = Input(shape=(1))
emb_month = Embedding(input_dim=12+1, output_dim=5)(i_month)
flat_month = Flatten()(emb_month)


#media hora
i_time = Input(shape=(1), name='i_time')

#ocup
i_ocup = Input(shape=(train_size,1), name='i_ocup')
lstm = LSTM(10, return_sequences=False, name='lstm')(i_ocup)


concat = Concatenate(axis=-1, name='input_merge')
input_merge = concat([i_porc_ocup, flat_street])
input_merge = concat([input_merge, i_time])
input_merge = concat([input_merge, flat_weekday])
input_merge = concat([input_merge, flat_month])
input_merge = concat([input_merge, lstm])

d = BatchNormalization()(input_merge)
d = Dense(256, activation='relu')(d)
d = Dropout(0.2)(d)
d = Dense(512, activation='relu')(d)
d = BatchNormalization()(d)
d = Dropout(0.2)(d)
d = Dense(512, activation='relu')(d)

o = Dense(max(data['porcentaje ocupacion'])+1, activation='softmax', name='Output')(d)

model = Model(inputs=[i_porc_ocup, i_street, i_weekday, i_month, i_time, i_ocup], outputs=o)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
i_street (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
emd_street (Embedding)          (None, 1, 10)        1910        i_street[0][0]                   
__________________________________________________________________________________________________
i_porc_ocup (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
flat_street (Flatten)           (None, 10)           0           emd_street[0][0]                 
___________________________________________________________________________________________

In [104]:
train_ocup_tf = tf.convert_to_tensor(np.array(train_ocup).reshape(-1,train_size,1), np.float64)
test_ocup_tf = tf.convert_to_tensor(np.array(test_ocup).reshape(-1,train_size,1), np.float64)

In [105]:
x_train = [np.array(train_porc_ocup), np.array(train_id_cuadra), np.array(train_dia_de_la_semana), np.array(train_mes), np.array(train_media_hora), train_ocup_tf]
x_test = [np.array(test_porc_ocup), np.array(test_id_cuadra), np.array(test_dia_de_la_semana), np.array(test_mes),  np.array(test_media_hora), test_ocup_tf]
#xt = [x_test_porc_ocup, x_test['id_cuadra'], x_test['media hora'], x_train['ocupacion']]
h = model.fit(x_train, train_target, epochs=10, batch_size=128, class_weight=class_weight, validation_data=[x_test, test_target])

Train on 330459 samples, validate on 36717 samples
Epoch 1/10
330459/330459 [==============================] - 41s 123us/sample - loss: 0.8395 - categorical_accuracy: 0.6387 - val_loss: 0.7887 - val_categorical_accuracy: 0.6602
Epoch 2/10
330459/330459 [==============================] - 35s 107us/sample - loss: 0.7914 - categorical_accuracy: 0.6568 - val_loss: 0.7691 - val_categorical_accuracy: 0.6623
Epoch 3/10
330459/330459 [==============================] - 35s 105us/sample - loss: 0.7807 - categorical_accuracy: 0.6607 - val_loss: 0.7630 - val_categorical_accuracy: 0.6655
Epoch 4/10
330459/330459 [==============================] - 34s 104us/sample - loss: 0.7752 - categorical_accuracy: 0.6630 - val_loss: 0.7529 - val_categorical_accuracy: 0.6722
Epoch 5/10
330459/330459 [==============================] - 32s 98us/sample - loss: 0.7707 - categorical_accuracy: 0.6644 - val_loss: 0.7529 - val_categorical_accuracy: 0.6683
Epoch 6/10
330459/330459 [==============================] - 34s 1

In [106]:
x_test = [tf.convert_to_tensor(x, np.float64) for x in x_test]

In [107]:
pred = [x.argmax() for x in model.predict(x_test)]
test_t = pd.Series([x.argmax() for x in test_target])

cnf_matrix = confusion_matrix(test_t, pred,labels=np.sort(test_t.unique()))
cnf_matrix

array([[9371, 3299,  121,    9,    0],
       [2410, 9586, 1740,   97,    5],
       [ 113, 2344, 4349,  550,    4],
       [   6,  185,  824, 1211,   53],
       [   1,   13,   38,  222,  166]], dtype=int64)

In [108]:
print(classification_report(test_t, pred))

              precision    recall  f1-score   support

           0       0.79      0.73      0.76     12800
           1       0.62      0.69      0.66     13838
           2       0.61      0.59      0.60      7360
           3       0.58      0.53      0.55      2279
           4       0.73      0.38      0.50       440

    accuracy                           0.67     36717
   macro avg       0.67      0.58      0.61     36717
weighted avg       0.68      0.67      0.67     36717

